# Importing libraries

In [1]:
import os
import sys
from dotenv import load_dotenv
from typing import Tuple
import torch
from torch.utils.data import Dataset, DataLoader
import wandb
from src.utils import set_seed, load_text, split_text, speedometer
from src.config import ModelConfig, TrainConfig, GenerationConfig
from src.tokenizer import CharTokenizer
from models.GPT import GPT
from src.train import Trainer

/home/pathfinder/miniconda3/envs/torch-env/lib/python3.12/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:
PROJECT_ROOT = os.path.abspath(os.getcwd())
sys.path.append(PROJECT_ROOT)
print(f"PROJECT_ROOT: {PROJECT_ROOT}")

PROJECT_ROOT: /home/pathfinder/projects/PathFinder


# Configuration

In [3]:
model_config = ModelConfig(
    vocab_size=-1,
    max_seq_len=128,
    d_embed=128,
    n_layers=4,
    n_heads=4,
    d_head=32,
    d_ff=512,
    rank=32
)

train_config = TrainConfig(
    debug=False,
    wandb_project="nanoGPT",
    model_name="nanoGPT",
    per_device_train_batch_size=512,
    per_device_eval_batch_size=1024,
    gradient_accumulation_steps=512 // 512,
    num_train_epochs=1,
    learning_rate=5e-3,
    eval_steps=100,
    mixed_precision=True,
    matmul_precision="high",
)

generation_config = GenerationConfig(
    use_cache=True,
    max_new_tokens=1000,
    temperature=1.0,
    top_k=50
)

In [4]:
load_dotenv()
wandb.login(key=os.environ.get("WANDB_API_KEY"))

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/pathfinder/.netrc
wandb: Currently logged in as: pathfinderkr to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

# Utils

## Reproducibility

In [5]:
set_seed(train_config.seed)

Random seed set to 42


## Device

In [6]:
device = torch.device("cuda")
print(f"Device: {torch.cuda.get_device_name(device)}")
torch.set_float32_matmul_precision(train_config.matmul_precision)  # Tensor Cores
print(f"MatMul Precision: {train_config.matmul_precision}")

Device: NVIDIA GeForce RTX 4080 SUPER
MatMul Precision: high


# Dataset

In [7]:
dataset_path = os.path.join(PROJECT_ROOT, "datasets/Shakespeare/shakespeare.txt")
shakespeare_text = load_text(dataset_path)

Loaded text data from /home/pathfinder/projects/PathFinder/datasets/Shakespeare/shakespeare.txt (length: 1115394 characters).


In [8]:
if train_config.debug:
    subset_shakespeare_text = shakespeare_text[:10000]
    print(subset_shakespeare_text)
    shakespeare_text = subset_shakespeare_text

# Tokenizer

In [9]:
char_tokenizer = CharTokenizer()
char_tokenizer.build_vocab(text=shakespeare_text)
vocab_path = os.path.join(PROJECT_ROOT, "datasets/Shakespeare/vocab.json")
char_tokenizer.save_vocab(vocab_path)
model_config.vocab_size = char_tokenizer.vocab_size

Vocabulary size: 69
Vocabulary saved to /home/pathfinder/projects/PathFinder/datasets/Shakespeare/vocab.json.


In [10]:
if train_config.debug:
    print("Vocabulary:", char_tokenizer.char2idx)

# Preprocessing

In [11]:
train_text, val_text = split_text(shakespeare_text, val_size=0.1)
print(f"Training text length: {len(train_text)} characters")
print(f"Validation text length: {len(val_text)} characters")

Training text length: 1003854 characters
Validation text length: 111540 characters


In [12]:
class TextDataset(Dataset):
    def __init__(self, text: str, tokenizer: CharTokenizer, max_seq_len: int):
        self.encoded = tokenizer.encode(text)
        self.max_seq_len = max_seq_len

    def __len__(self) -> int:
        return len(self.encoded) - self.max_seq_len

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        input_ids = self.encoded[idx:idx + self.max_seq_len]
        target_ids = self.encoded[idx + 1:idx + self.max_seq_len + 1]
        return input_ids, target_ids

def collate_fn(batch):
    input_ids = torch.stack([item[0] for item in batch])
    target_ids = torch.stack([item[1] for item in batch])
    return {"input_ids": input_ids, "target_ids": target_ids}

train_dataset = TextDataset(train_text, char_tokenizer, model_config.max_seq_len)
val_dataset = TextDataset(val_text, char_tokenizer, model_config.max_seq_len)

train_loader = DataLoader(
    train_dataset,
    collate_fn=collate_fn,
    batch_size=train_config.per_device_eval_batch_size,
    shuffle=True,
    num_workers=4
)
val_loader = DataLoader(
    val_dataset,
    collate_fn=collate_fn,
    batch_size=train_config.per_device_eval_batch_size,
    shuffle=False,
    num_workers=4
)

In [13]:
if train_config.debug:
    sample_batch = next(iter(train_loader))
    print(f"Sample input IDs: {sample_batch['input_ids'][0]}")
    print(f"Sample target IDs: {sample_batch['target_ids'][0]}")

# Model

In [14]:
# Initialize the model
model = GPT(model_config).to(device)
model = torch.compile(model)
print(model)
print(f"Number of parameters: {model.num_params() / 1e6:.2f}M")

OptimizedModule(
  (_orig_mod): GPT(
    (token_embedding): Embedding(69, 128)
    (positional_encoding): Embedding(128, 128)
    (dropout): Dropout(p=0.1, inplace=False)
    (blocks): ModuleList(
      (0-3): 4 x Block(
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (attn): MultiHeadAttention(
          (Wq): Linear(in_features=128, out_features=128, bias=False)
          (Wkv_down): Linear(in_features=128, out_features=32, bias=False)
          (Wkv_up): Linear(in_features=32, out_features=256, bias=False)
          (Wk_up): Linear(in_features=32, out_features=128, bias=False)
          (Wv_up): Linear(in_features=32, out_features=128, bias=False)
          (out_proj): Linear(in_features=128, out_features=128, bias=False)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (mlp): FeedForward(
          (fc1): Linear(in_features=128, out_features=512, bias=False)
   

# Training

In [15]:
trainer = Trainer(
    model=model,
    train_config=train_config,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    master_process=True
)
trainer.train()

Training:   0%|          | 0/981 [00:00<?, ?it/s]

k shape: torch.Size([1024, 128, 128])
v shape: torch.Size([1024, 128, 128])
k shape: torch.Size([1024, 128, 128])
v shape: torch.Size([1024, 128, 128])



class GraphModule(torch.nn.Module):
    def forward(self, L_kv_latent_: "bf16[1024, 128, 32][4096, 32, 1]cuda:0"):
        l_kv_latent_ = L_kv_latent_
        


k shape: torch.Size([1024, 128, 128])
v shape: torch.Size([1024, 128, 128])
k shape: torch.Size([1024, 128, 128])
v shape: torch.Size([1024, 128, 128])


Training:   0%|          | 4/981 [00:00<-1:59:52, -109.76it/s, epoch=1, grad_norm=1.5479, loss=4.6219, lr=0.000255]

...

Training: 100%|██████████| 981/981 [01:05<00:00,  5.06it/s, epoch=1, grad_norm=0.2001, loss=1.3565, lr=0.000000]

k shape: torch.Size([206, 128, 128])
v shape: torch.Size([206, 128, 128])
k shape: torch.Size([206, 128, 128])
v shape: torch.Size([206, 128, 128])
k shape: torch.Size([206, 128, 128])
v shape: torch.Size([206, 128, 128])
k shape: torch.Size([1024, 128, 128])
v shape: torch.Size([1024, 128, 128])
k shape: torch.Size([1024, 128, 128])
v shape: torch.Size([1024, 128, 128])
k shape: torch.Size([1024, 128, 128])
v shape: torch.Size([1024, 128, 128])
k shape: torch.Size([1024, 128, 128])
v shape: torch.Size([1024, 128, 128])
k shape: torch.Size([1024, 128, 128])
v shape: torch.Size([1024, 128, 128])
k shape: torch.Size([1024, 128, 128])
v shape: torch.Size([1024, 128, 128])
k shape: torch.Size([1024, 128, 128])
v shape: torch.Size([1024, 128, 128])
k shape: torch.Size([1024, 128, 128])
v shape: torch.Size([1024, 128, 128])
k shape: torch.Size([1024, 128, 128])
v shape: torch.Size([1024, 128, 128])
k shape: torch.Size([1024, 128, 128])
v shape: torch.Size([1024, 128, 128])
k shape: torch.Siz

Training: 100%|██████████| 981/981 [01:07<00:00, 14.56it/s, epoch=1, grad_norm=0.2001, loss=1.3565, lr=0.000000]

k shape: torch.Size([1024, 128, 128])
v shape: torch.Size([1024, 128, 128])
k shape: torch.Size([1024, 128, 128])
v shape: torch.Size([1024, 128, 128])
k shape: torch.Size([1024, 128, 128])
v shape: torch.Size([1024, 128, 128])
k shape: torch.Size([1024, 128, 128])
v shape: torch.Size([1024, 128, 128])
k shape: torch.Size([1024, 128, 128])
v shape: torch.Size([1024, 128, 128])
k shape: torch.Size([1024, 128, 128])
v shape: torch.Size([1024, 128, 128])
k shape: torch.Size([1024, 128, 128])
v shape: torch.Size([1024, 128, 128])
k shape: torch.Size([1024, 128, 128])
v shape: torch.Size([1024, 128, 128])
k shape: torch.Size([1024, 128, 128])
v shape: torch.Size([1024, 128, 128])
k shape: torch.Size([1024, 128, 128])
v shape: torch.Size([1024, 128, 128])
k shape: torch.Size([1024, 128, 128])
v shape: torch.Size([1024, 128, 128])
k shape: torch.Size([1024, 128, 128])
v shape: torch.Size([1024, 128, 128])
k shape: torch.Size([820, 128, 128])
v shape: torch.Size([820, 128, 128])
k shape: torch

Grad Norm,█▃▃█▄▃▃▅▄▃▅▅▂▂▂▂▂▃▂▂▂▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁
Learning Rate,▁▂▃▄▆█████▇▇▇▆▆▅▅▅▅▅▄▄▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
Train Loss,█▆▆▆▆▅▅▅▅▅▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Val Loss,█▅▃▂▂▁▁▁▁▁
Val Perplexity,█▄▂▂▁▁▁▁▁▁
Grad Norm,0.20014
Learning Rate,0
Train Loss,1.35646
Val Loss,1.49788
Val Perplexity,4.47221


## Save the model

In [16]:
# Save model locally
if not train_config.debug:
    output_dir = os.path.join(PROJECT_ROOT, "checkpoints", train_config.model_name, train_config.run_name)
    os.makedirs(output_dir, exist_ok=True)
    try:
        model.save_pretrained(
            output_dir,
            safe_serialization=True
        )
        print("Model saved successfully.")
    except Exception as e:
        print(f"Error saving model: {e}")
    # Push to Hugging Face Hub
    #model.push_to_hub(
    #    repo_id=f"PathFinderKR/{train_config.model_name}-{train_config.run_name}",
    #    private=True,
    #    use_auth_token=os.environ.get("HUGGINGFACE_TOKEN")
    #)
    #print(f"Model pushed to Hugging Face Hub: PathFinderKR/{train_config.model_name}-{train_config.run_name}")

Model saved successfully.


In [17]:
# To load the model later, you can use:
# model = GPT(model_config)
# model = model.from_pretrained(output_dir).to(device)

# Inference

In [18]:
user_prompt = "To be, or not to be, that is the question"
input_ids = char_tokenizer.encode(user_prompt).unsqueeze(0).to(device)
output = model.generate(
    input_ids,
    use_cache=True,
    max_new_tokens=generation_config.max_new_tokens,
    temperature=generation_config.temperature,
    top_k=generation_config.top_k,
    tokenizer=char_tokenizer
)
response = char_tokenizer.decode(output[0].squeeze().cpu().numpy())

k shape: torch.Size([1, 41, 128])
v shape: torch.Size([1, 41, 128])
k shape: torch.Size([1, 41, 128])
v shape: torch.Size([1, 41, 128])
k shape: torch.Size([1, 41, 128])
v shape: torch.Size([1, 41, 128])
k shape: torch.Size([1, 41, 128])
v shape: torch.Size([1, 41, 128])
,kv_latent shape: torch.Size([1, 42, 32])
k shape: torch.Size([1, 42, 128])
v shape: torch.Size([1, 42, 128])
kv_latent shape: torch.Size([1, 42, 32])
k shape: torch.Size([1, 42, 128])
v shape: torch.Size([1, 42, 128])
kv_latent shape: torch.Size([1, 42, 32])
k shape: torch.Size([1, 42, 128])
v shape: torch.Size([1, 42, 128])
kv_latent shape: torch.Size([1, 42, 32])
k shape: torch.Size([1, 42, 128])
v shape: torch.Size([1, 42, 128])

kv_latent shape: torch.Size([1, 43, 32])
k shape: torch.Size([1, 43, 128])
v shape: torch.Size([1, 43, 128])
kv_latent shape: torch.Size([1, 43, 32])
k shape: torch.Size([1, 43, 128])
v shape: torch.Size([1, 43, 128])
kv_latent shape: torch.Size([1, 43, 32])
k shape: torch.Size([1, 43, 128

In [19]:
print("=" * 50)
print("User prompt: ")
print(user_prompt)
print("-" * 50)
print("🤖 Model Response:")
print(response)

User prompt: 
To be, or not to be, that is the question
--------------------------------------------------
🤖 Model Response:
To be, or not to be, that is the question,
Not to this friend: this your victor'd,
Sweet thee that by age give what old by the fr, that is and tell you to that?

ROMEO:
I have you born me azon than all the very
Mice, or dance be condit as show'd for your hin.

Clown:
Well, a bed a but wife in this view not
Trepution most honour for your son guard:
And if they is ever argue to me be well--thoughts means to thear, is return'd
All dry else force! 'Tis further, as my lord!
Ay, as, the another of another'd, as I
She was too strike a cause of two hand.

KING RICHARD II:
Because Benop, let thee to, I have a voice
Made that for my flesh of ast the Earll, of York over
That they have reverenced, lauding and wars
Was weak'd, speaking from the office
As variant to circumes
No more:--by my! Come, my lord. What, be't no
my body is mother of your breath condition?
He dog, call 

# Speedometer

In [20]:
speedometer(
    model=model,
    input_ids=char_tokenizer.encode("a").unsqueeze(0).to(device),
    use_cache=True,
    warmup_tokens=100,
    timing_tokens=100,
    num_runs=5
)

k shape: torch.Size([1, 1, 128])
v shape: torch.Size([1, 1, 128])
k shape: torch.Size([1, 1, 128])
v shape: torch.Size([1, 1, 128])
k shape: torch.Size([1, 1, 128])
v shape: torch.Size([1, 1, 128])
k shape: torch.Size([1, 1, 128])
v shape: torch.Size([1, 1, 128])
kv_latent shape: torch.Size([1, 2, 32])
k shape: torch.Size([1, 2, 128])
v shape: torch.Size([1, 2, 128])
kv_latent shape: torch.Size([1, 2, 32])
k shape: torch.Size([1, 2, 128])
v shape: torch.Size([1, 2, 128])
kv_latent shape: torch.Size([1, 2, 32])
k shape: torch.Size([1, 2, 128])
v shape: torch.Size([1, 2, 128])
kv_latent shape: torch.Size([1, 2, 32])
k shape: torch.Size([1, 2, 128])
v shape: torch.Size([1, 2, 128])
kv_latent shape: torch.Size([1, 3, 32])
k shape: torch.Size([1, 3, 128])
v shape: torch.Size([1, 3, 128])
kv_latent shape: torch.Size([1, 3, 32])
k shape: torch.Size([1, 3, 128])
v shape: torch.Size([1, 3, 128])
kv_latent shape: torch.Size([1, 3, 32])
k shape: torch.Size([1, 3, 128])
v shape: torch.Size([1, 3, 

In [21]:
speedometer(
    model=model,
    input_ids=char_tokenizer.encode("a").unsqueeze(0).to(device),
    use_cache=False,
    warmup_tokens=100,
    timing_tokens=100,
    num_runs=5
)

k shape: torch.Size([1, 1, 128])
v shape: torch.Size([1, 1, 128])
k shape: torch.Size([1, 1, 128])
v shape: torch.Size([1, 1, 128])
k shape: torch.Size([1, 1, 128])
v shape: torch.Size([1, 1, 128])
k shape: torch.Size([1, 1, 128])
v shape: torch.Size([1, 1, 128])
k shape: torch.Size([1, 2, 128])
v shape: torch.Size([1, 2, 128])
k shape: torch.Size([1, 2, 128])
v shape: torch.Size([1, 2, 128])
k shape: torch.Size([1, 2, 128])
v shape: torch.Size([1, 2, 128])
k shape: torch.Size([1, 2, 128])
v shape: torch.Size([1, 2, 128])
k shape: torch.Size([1, 3, 128])
v shape: torch.Size([1, 3, 128])
k shape: torch.Size([1, 3, 128])
v shape: torch.Size([1, 3, 128])
k shape: torch.Size([1, 3, 128])
v shape: torch.Size([1, 3, 128])
k shape: torch.Size([1, 3, 128])
v shape: torch.Size([1, 3, 128])
k shape: torch.Size([1, 4, 128])
v shape: torch.Size([1, 4, 128])
k shape: torch.Size([1, 4, 128])
v shape: torch.Size([1, 4, 128])
k shape: torch.Size([1, 4, 128])
v shape: torch.Size([1, 4, 128])
k shape: t